In [1]:
import sqlite3
import logging
import os

# --- Configuration ---
DB_FILE = "prices.db"
LOG_FILE = "setup_db.log"

# --- Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler() # Also print to console
    ])

# --- Database Schema Definition ---

# SQL statement for the 'prices' table
CREATE_PRICES_TABLE = """
CREATE TABLE IF NOT EXISTS prices (
    caption TEXT PRIMARY KEY,    -- Unique name of the price item (e.g., "سکه امامی")
    value REAL NOT NULL,         -- Latest processed price value
    timestamp INTEGER NOT NULL   -- Unix timestamp of the last update
);
"""

# SQL statement for the 'users' table
CREATE_USERS_TABLE = """
CREATE TABLE IF NOT EXISTS users (
    chat_id INTEGER PRIMARY KEY, -- Telegram user's unique chat ID
    username TEXT,               -- Telegram username (nullable)
    first_name TEXT,             -- Telegram first name (nullable)
    last_name TEXT,              -- Telegram last name (nullable)
    last_message_id INTEGER      -- ID of the last price update message sent (nullable)
);
"""

# SQL statement for the 'subscriptions' table
CREATE_SUBSCRIPTIONS_TABLE = """
CREATE TABLE IF NOT EXISTS subscriptions (
    chat_id INTEGER NOT NULL,   -- Foreign key to the users table
    caption TEXT NOT NULL,      -- The specific price item the user subscribed to

    -- Define composite primary key to ensure user subscribes to an item only once
    PRIMARY KEY (chat_id, caption),

    -- Define foreign key constraint: if a user is deleted, their subscriptions are also deleted
    FOREIGN KEY (chat_id) REFERENCES users(chat_id) ON DELETE CASCADE
);
"""

# Optional: Create an index for faster subscription lookups by chat_id
CREATE_SUBSCRIPTION_INDEX = """
CREATE INDEX IF NOT EXISTS idx_sub_chat_id ON subscriptions (chat_id);
"""

def create_database_schema():
    """
    Connects to the SQLite database (creating it if it doesn't exist)
    and executes the CREATE TABLE statements.
    """
    logging.info(f"Attempting to set up database schema in '{DB_FILE}'...")
    conn = None # Initialize connection variable
    try:
        # Connect to the database. Creates the file if it doesn't exist.
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        logging.info(f"Database connection established to '{DB_FILE}'.")

        # Enable Foreign Key constraint enforcement (important!)
        cursor.execute("PRAGMA foreign_keys = ON;")
        logging.info("Foreign key enforcement enabled.")

        # Execute table creation statements
        logging.info("Creating 'prices' table (if not exists)...")
        cursor.execute(CREATE_PRICES_TABLE)

        logging.info("Creating 'users' table (if not exists)...")
        cursor.execute(CREATE_USERS_TABLE)

        logging.info("Creating 'subscriptions' table (if not exists)...")
        cursor.execute(CREATE_SUBSCRIPTIONS_TABLE)

        logging.info("Creating index on 'subscriptions.chat_id' (if not exists)...")
        cursor.execute(CREATE_SUBSCRIPTION_INDEX)

        # Commit the changes
        conn.commit()
        logging.info("Database schema created/verified successfully.")

    except sqlite3.Error as e:
        logging.error(f"Database setup failed: {e}")
        # Rollback changes if any error occurred during transaction
        if conn:
            conn.rollback()
            logging.info("Database transaction rolled back due to error.")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()
            logging.info("Database transaction rolled back due to unexpected error.")
    finally:
        # Ensure the connection is closed even if errors occur
        if conn:
            conn.close()
            logging.info("Database connection closed.")

# --- Main Execution ---
if __name__ == "__main__":
    # Check if the DB file already exists (optional, for info)
    if os.path.exists(DB_FILE):
        logging.info(f"Database file '{DB_FILE}' already exists. Schema will be verified/updated.")
    else:
        logging.info(f"Database file '{DB_FILE}' not found. It will be created.")

    # Run the setup function
    create_database_schema()

2025-04-03 12:32:24,738 - INFO - Database file 'prices.db' not found. It will be created.
2025-04-03 12:32:24,739 - INFO - Attempting to set up database schema in 'prices.db'...
2025-04-03 12:32:24,742 - INFO - Database connection established to 'prices.db'.
2025-04-03 12:32:24,743 - INFO - Foreign key enforcement enabled.
2025-04-03 12:32:24,744 - INFO - Creating 'prices' table (if not exists)...
2025-04-03 12:32:24,749 - INFO - Creating 'users' table (if not exists)...
2025-04-03 12:32:24,760 - INFO - Creating 'subscriptions' table (if not exists)...
2025-04-03 12:32:24,767 - INFO - Creating index on 'subscriptions.chat_id' (if not exists)...
2025-04-03 12:32:24,773 - INFO - Database schema created/verified successfully.
2025-04-03 12:32:24,776 - INFO - Database connection closed.


In [9]:
import requests
import schedule
import time
import sqlite3
import logging
import os
from datetime import datetime

# --- Configuration ---
API_URL = "http://et.tala.ir/webservice/rafshan/6397db883000095bb8ed65398865c994"
DB_FILE = "prices.db"
LOG_FILE = "price_miner.log"
FETCH_INTERVAL_MINUTES = 1

# --- Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler() # Also print to console
    ])

# --- Database Setup ---
def setup_database():
    """Creates the database and prices table if they don't exist."""
    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            # Prices table: Stores the latest processed price for each item
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS prices (
                caption TEXT PRIMARY KEY,
                value REAL NOT NULL,
                timestamp INTEGER NOT NULL
            )
            """)
            conn.commit()
            logging.info("Database setup complete.")
    except sqlite3.Error as e:
        logging.error(f"Database setup error: {e}")
        raise # Reraise to stop the script if DB setup fails

# --- Price Fetching and Processing ---
def fetch_prices():
    """Fetches raw price data from the API."""
    try:
        response = requests.get(API_URL, timeout=10) # Add timeout
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching API data: {e}")
        return None

def process_prices(api_data):
    """Processes raw data, applies multipliers, and returns final values."""
    if not api_data:
        return {}

    processed = {}
    for key, item_data in api_data.items():
        try:
            if item_data and 'caption' in item_data and 'value' in item_data:
                caption = item_data['caption']
                raw_value = float(item_data['value'])

                if caption is None or not isinstance(caption, str) or caption.strip() == "":
                    logging.warning(f"Skipping item with invalid caption: {item_data}")
                    continue

                # Apply conditional multipliers
                if "انس" in caption:  # Check for "ounce"
                    processed_value = raw_value * 10
                else:
                    processed_value = raw_value * 0.1

                processed[caption] = processed_value
            else:
                 logging.warning(f"Skipping invalid item data: Key='{key}', Data='{item_data}'")

        except (ValueError, TypeError) as e:
            logging.warning(f"Could not process item: Key='{key}', Data='{item_data}'. Error: {e}")
            continue # Skip this item if value conversion fails

    return processed

# --- Database Storage ---
def store_prices(processed_prices):
    """Stores processed prices into the SQLite database."""
    if not processed_prices:
        logging.info("No processed prices to store.")
        return

    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            timestamp = int(datetime.now().timestamp()) # Use Unix timestamp

            for caption, value in processed_prices.items():
                # Use INSERT OR REPLACE to add new or update existing prices
                cursor.execute("""
                INSERT OR REPLACE INTO prices (caption, value, timestamp)
                VALUES (?, ?, ?)
                """, (caption, value, timestamp))

            conn.commit()
            logging.info(f"Stored/Updated {len(processed_prices)} prices in the database.")

    except sqlite3.Error as e:
        logging.error(f"Database error during price storage: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred during price storage: {e}")


# --- Main Job ---
def price_update_job():
    """The main job to be scheduled."""
    logging.info("Running price update job...")
    raw_data = fetch_prices()
    if raw_data:
        processed_data = process_prices(raw_data)
        store_prices(processed_data)
    logging.info("Price update job finished.")

# --- Scheduler ---
if __name__ == "__main__":
    logging.info("Starting Price Miner App...")
    setup_database()

    # Schedule the job
    schedule.every(FETCH_INTERVAL_MINUTES).minutes.do(price_update_job)

    # Run the job immediately first time
    price_update_job()

    # Keep the script running to execute scheduled jobs
    while True:
        schedule.run_pending()
        time.sleep(1)

2025-04-05 00:15:53,865 - INFO - Starting Price Miner App...
2025-04-05 00:15:53,880 - INFO - Database setup complete.
2025-04-05 00:15:53,883 - INFO - Running price update job...
2025-04-05 00:15:55,564 - WARNING - Skipping invalid item data: Key='photo', Data=''
2025-04-05 00:15:55,565 - WARNING - Could not process item: Key='timestamp', Data='1743800548'. Error: argument of type 'int' is not iterable
2025-04-05 00:15:55,567 - WARNING - Skipping invalid item data: Key='news', Data='[{'title': '   سکه شرق - سیاست شفاف در ارائه نرخ خرید و فروش - دارای مجوز از اتحادیه طلا و جواهر تهران - مقدم شما را گرامی می دارد    '}]'
2025-04-05 00:15:55,570 - WARNING - Skipping invalid item data: Key='expire', Data='2025-06-10'
2025-04-05 00:15:55,571 - WARNING - Skipping invalid item data: Key='serverTime', Data='2025-04-05 01:32:28'
2025-04-05 00:15:55,583 - INFO - Stored/Updated 16 prices in the database.
2025-04-05 00:15:55,586 - INFO - Price update job finished.
2025-04-05 00:15:55,590 - INFO -

KeyboardInterrupt: 

In [7]:
import logging
import sqlite3
import os
import jdatetime # For Shamsi date
from datetime import datetime
import asyncio # For potential sleeps if needed

from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (
    Application,
    CommandHandler,
    CallbackQueryHandler,
    ContextTypes,
    PersistenceInput, # For user_data persistence (optional but good)
    DictPersistence,  # Simple dictionary-based persistence
)
from telegram.constants import ParseMode
from telegram.error import BadRequest, Forbidden # Specific Telegram errors

# --- Configuration ---
TELEGRAM_TOKEN = "7870608369:AAEtxEq3DYQgfMCmwMPO_E_dhhAk56SpNow" # Replace with your token
DB_FILE = "prices.db"
LOG_FILE = "telegram_bot.log"
UPDATE_INTERVAL_SECONDS = 60 # Send updates every 60 seconds

# --- Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler()
    ])
logger = logging.getLogger(__name__)

# --- Callback Data Constants ---
CALLBACK_DONE = "DONE_SELECT"
CALLBACK_PREFIX_TOGGLE = "TOGGLE_"

# --- Database Setup & Helpers ---
def setup_database():
    """Creates user-related tables if they don't exist."""
    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            # Users table: Basic user info + last message ID for updates
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
                chat_id INTEGER PRIMARY KEY,
                username TEXT,
                first_name TEXT,
                last_name TEXT,
                last_message_id INTEGER
            )""")
            # Subscriptions table: Links users to the price captions they follow
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS subscriptions (
                chat_id INTEGER NOT NULL,
                caption TEXT NOT NULL,
                FOREIGN KEY (chat_id) REFERENCES users(chat_id),
                PRIMARY KEY (chat_id, caption)
            )""")
            # Optional: Index for faster lookups if needed later
            # cursor.execute("CREATE INDEX IF NOT EXISTS idx_sub_chat_id ON subscriptions (chat_id)")
            conn.commit()
            logger.info("User database setup complete.")
    except sqlite3.Error as e:
        logger.error(f"User database setup error: {e}")
        raise

def db_query(query, params=(), fetchone=False, commit=False):
    """General purpose DB query helper to reduce boilerplate."""
    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute(query, params)
            if commit:
                conn.commit()
                return None # Or return affected rows if needed
            else:
                return cursor.fetchone() if fetchone else cursor.fetchall()
    except sqlite3.Error as e:
        logger.error(f"Database query error. Query: '{query}', Params: {params}. Error: {e}")
        # Depending on severity, you might want to return None or raise
        return None if fetchone else [] # Return empty list/None on error for reads

# --- User & Subscription Management ---
async def register_user(chat_id: int, username: str, first_name: str, last_name: str):
    """Adds or ignores a user in the users table."""
    query = """
    INSERT OR IGNORE INTO users (chat_id, username, first_name, last_name, last_message_id)
    VALUES (?, ?, ?, ?, NULL)
    """
    db_query(query, (chat_id, username, first_name, last_name), commit=True)
    logger.info(f"Registered/Verified user: {chat_id} ({username or 'No username'})")

def get_user_subscriptions(chat_id: int) -> list[str]:
    """Retrieves the list of captions a user is subscribed to."""
    query = "SELECT caption FROM subscriptions WHERE chat_id = ?"
    results = db_query(query, (chat_id,))
    return [row[0] for row in results]

def get_available_items() -> list[str]:
    """Gets distinct available price captions from the database."""
    query = "SELECT DISTINCT caption FROM prices ORDER BY caption"
    results = db_query(query)
    if not results:
        logger.warning("No price items found in the database. Miner might not be running.")
    return [row[0] for row in results]

def update_user_subscriptions(chat_id: int, captions: list[str]):
    """Updates a user's subscriptions atomically."""
    delete_query = "DELETE FROM subscriptions WHERE chat_id = ?"
    insert_query = "INSERT INTO subscriptions (chat_id, caption) VALUES (?, ?)"

    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            # Start transaction
            cursor.execute("BEGIN TRANSACTION")
            # Delete old subscriptions
            cursor.execute(delete_query, (chat_id,))
            # Insert new ones
            if captions:
                cursor.executemany(insert_query, [(chat_id, caption) for caption in captions])
            # Commit transaction
            cursor.execute("COMMIT")
            logger.info(f"Updated subscriptions for {chat_id}. New count: {len(captions)}")
    except sqlite3.Error as e:
        logger.error(f"Database error updating subscriptions for {chat_id}: {e}")
        try:
             conn.execute("ROLLBACK") # Rollback on error
        except: pass # Ignore rollback errors

# --- Telegram Command Handlers ---
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handles the /start command."""
    user = update.effective_user
    if not user:
        logger.warning("Received /start but could not get user info.")
        return

    chat_id = user.id
    username = user.username
    first_name = user.first_name
    last_name = user.last_name

    await register_user(chat_id, username, first_name, last_name)

    # Initialize or clear temporary selection state in user_data
    context.user_data['temp_selection'] = set(get_user_subscriptions(chat_id)) # Pre-load current subs

    await send_item_selection_keyboard(chat_id, context, "سلام! لطفا موارد مورد نظر برای دریافت قیمت را انتخاب کنید:")

# --- Telegram Callback Query Handler ---
async def handle_callback_query(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handles button presses from inline keyboards."""
    query = update.callback_query
    await query.answer() # Acknowledge callback quickly

    chat_id = query.message.chat_id
    message_id = query.message.message_id
    callback_data = query.data

    # Ensure temporary selection exists
    if 'temp_selection' not in context.user_data:
        context.user_data['temp_selection'] = set(get_user_subscriptions(chat_id)) # Initialize if missing


    if callback_data == CALLBACK_DONE:
        # --- User finished selection ---
        final_selection_set = context.user_data.get('temp_selection', set())
        final_selection_list = sorted(list(final_selection_set)) # Save sorted list

        update_user_subscriptions(chat_id, final_selection_list)
        del context.user_data['temp_selection'] # Clean up temp data

        confirmation_text = "✅ تنظیمات شما ذخیره شد.\n قیمت موارد زیر برای شما ارسال خواهد شد:\n\n"
        if final_selection_list:
            confirmation_text += "- " + "\n- ".join(final_selection_list)
        else:
            confirmation_text += "<i>هیچ موردی انتخاب نشده است.</i>"

        try:
            await context.bot.edit_message_text(
                chat_id=chat_id,
                message_id=message_id,
                text=confirmation_text,
                parse_mode=ParseMode.HTML,
                reply_markup=None # Remove keyboard
            )
        except BadRequest as e:
            logger.warning(f"Could not edit message after selection for {chat_id} (maybe unchanged?): {e}")


    elif callback_data.startswith(CALLBACK_PREFIX_TOGGLE):
        # --- User toggled an item ---
        item_caption = callback_data[len(CALLBACK_PREFIX_TOGGLE):]
        temp_selection_set = context.user_data.get('temp_selection', set())

        if item_caption in temp_selection_set:
            temp_selection_set.remove(item_caption)
        else:
            temp_selection_set.add(item_caption)

        context.user_data['temp_selection'] = temp_selection_set # Save updated set

        # Update the keyboard message
        await edit_selection_keyboard(chat_id, message_id, context, "لطفا موارد مورد نظر را انتخاب کنید:")

# --- Keyboard Generation & Sending ---
def build_selection_keyboard(available_items: list[str], selected_items: set[str]) -> InlineKeyboardMarkup:
    """Builds the dynamic inline keyboard for item selection."""
    keyboard = []
    row = []
    items_per_row = 2

    for item in available_items:
        is_selected = item in selected_items
        button_text = ("✅ " if is_selected else "") + item
        row.append(InlineKeyboardButton(button_text, callback_data=CALLBACK_PREFIX_TOGGLE + item))
        if len(row) >= items_per_row:
            keyboard.append(row)
            row = []
    if row:
        keyboard.append(row)

    keyboard.append([InlineKeyboardButton("✅ ذخیره و پایان", callback_data=CALLBACK_DONE)])
    return InlineKeyboardMarkup(keyboard)

async def send_item_selection_keyboard(chat_id: int, context: ContextTypes.DEFAULT_TYPE, text: str):
    """Sends the initial selection keyboard message."""
    available_items = get_available_items()
    if not available_items:
        await context.bot.send_message(chat_id, "متاسفانه در حال حاضر لیستی برای انتخاب وجود ندارد. لطفا بعدا دوباره /start را بزنید.")
        return

    selected_items_set = context.user_data.get('temp_selection', set())
    reply_markup = build_selection_keyboard(available_items, selected_items_set)

    await context.bot.send_message(
        chat_id=chat_id,
        text=text,
        reply_markup=reply_markup,
        parse_mode=ParseMode.HTML
    )

async def edit_selection_keyboard(chat_id: int, message_id: int, context: ContextTypes.DEFAULT_TYPE, text: str):
    """Edits the keyboard message during selection."""
    available_items = get_available_items() # Might need to refresh if items change? Unlikely mid-selection.
    selected_items_set = context.user_data.get('temp_selection', set())
    reply_markup = build_selection_keyboard(available_items, selected_items_set)

    try:
        await context.bot.edit_message_text( # Edit text too, in case instructions change
            chat_id=chat_id,
            message_id=message_id,
            text=text,
            reply_markup=reply_markup,
            parse_mode=ParseMode.HTML
        )
    except BadRequest as e:
        # Ignore if message is not modified
        if "Message is not modified" not in str(e):
            logger.warning(f"Failed to edit selection keyboard for {chat_id}: {e}")

# --- Price Update Sending Job ---
def get_shamsi_date() -> str:
    """Gets the current Shamsi date."""
    try:
        # Use jdatetime library
        now_gregorian = datetime.now()
        now_shamsi = jdatetime.datetime.fromgregorian(datetime=now_gregorian)
        # Format: YYYY/MM/DD (adjust as needed)
        return now_shamsi.strftime("%Y/%m/%d")
        # Alternative: Call the keybit API if preferred
        # response = requests.get("https://api.keybit.ir/time/")
        # response.raise_for_status()
        # return response.json()['date']['far']
    except Exception as e:
        logger.error(f"Failed to get Shamsi date: {e}")
        return "N/A"

def get_current_prices(captions: list[str]) -> dict[str, tuple[float, int]]:
    """Fetches current values and timestamps for specific captions from DB."""
    if not captions:
        return {}

    placeholders = ','.join('?' * len(captions))
    query = f"SELECT caption, value, timestamp FROM prices WHERE caption IN ({placeholders})"
    results = db_query(query, tuple(captions))

    # Create a dictionary for easy lookup
    price_dict = {row[0]: (row[1], row[2]) for row in results} # {caption: (value, timestamp)}
    return price_dict

async def send_updates_job(context: ContextTypes.DEFAULT_TYPE):
    """Job function run by JobQueue to send updates."""
    logger.info("Running scheduled update job...")

    # 1. Get all users who have subscriptions
    query_users = """
    SELECT DISTINCT u.chat_id, u.last_message_id
    FROM users u JOIN subscriptions s ON u.chat_id = s.chat_id
    """
    users_to_update = db_query(query_users)

    if not users_to_update:
        logger.info("No users with active subscriptions found.")
        return

    shamsi_date = get_shamsi_date()
    time_str = datetime.now().strftime("%H:%M:%S")
    message_footer = f"\n\n📡 <i>قیمت‌ها بروز هستند.</i>" # Simplified footer

    for chat_id, last_message_id in users_to_update:
        # 2. Get user's specific subscriptions
        user_subscriptions = get_user_subscriptions(chat_id)
        if not user_subscriptions:
            continue # Should not happen based on query_users, but check anyway

        # 3. Get current prices for subscribed items
        current_prices = get_current_prices(user_subscriptions)
        if not current_prices:
            logger.warning(f"No current prices found for subscriptions of user {chat_id}. Skipping.")
            continue # Skip user if their items aren't in the prices table

        # 4. Format message (No price comparison emoji here, just latest)
        message_body = ""
        has_data = False
        # Sort by caption maybe?
        for caption in sorted(user_subscriptions):
            if caption in current_prices:
                value, timestamp = current_prices[caption]
                # You could add logic here to compare with a 'previous_prices' cache
                # stored perhaps in context.bot_data if you want the 📈/📉 emojis back.
                # For simplicity now, just show the current value.
                message_body += f"🔹 <b>{caption}:</b> {value:,.0f} تومان\n\n" # Format as integer تومان
                has_data = True

        if not has_data:
            logger.info(f"No relevant price data found for user {chat_id} this cycle.")
            continue

        # 5. Construct and send/edit message
        message_header = f"📢 <b>آخرین قیمت‌ها (موارد انتخابی شما)</b>\n🗓 تاریخ: <b>{shamsi_date}</b>\n⏰ ساعت: <b>{time_str}\n\n"
        full_message = message_header + message_body.strip() + message_footer

        new_message_id = None
        try:
            if last_message_id:
                await context.bot.edit_message_text(
                    chat_id=chat_id,
                    message_id=last_message_id,
                    text=full_message,
                    parse_mode=ParseMode.HTML
                )
                logger.info(f"Edited message {last_message_id} for user {chat_id}")
            else:
                # Send new message if no previous ID
                sent_msg = await context.bot.send_message(
                    chat_id=chat_id,
                    text=full_message,
                    parse_mode=ParseMode.HTML
                )
                new_message_id = sent_msg.message_id
                logger.info(f"Sent new message {new_message_id} to user {chat_id}")

        except BadRequest as e:
            if "Message to edit not found" in str(e) or "message can't be edited" in str(e) or "Message is not modified" in str(e):
                 logger.warning(f"Editing failed for user {chat_id}, msg_id {last_message_id}. Sending new message. Error: {e}")
                 try:
                     sent_msg = await context.bot.send_message(chat_id=chat_id, text=full_message, parse_mode=ParseMode.HTML)
                     new_message_id = sent_msg.message_id
                     logger.info(f"Sent new message {new_message_id} after edit failed for user {chat_id}")
                 except Exception as send_e:
                     logger.error(f"Failed to send new message to user {chat_id} after edit failure: {send_e}")
            else:
                 logger.error(f"Unhandled BadRequest sending/editing update for {chat_id}: {e}")
        except Forbidden as e:
            logger.warning(f"Bot blocked or kicked by user {chat_id}: {e}. Consider removing user/subs.")
            # Add logic here to potentially remove the user's subscriptions if blocked
        except Exception as e:
             logger.error(f"Unexpected error sending update to user {chat_id}: {e}")

        # 6. Update last_message_id in DB if a new message was sent
        if new_message_id:
             db_query("UPDATE users SET last_message_id = ? WHERE chat_id = ?", (new_message_id, chat_id), commit=True)


# --- Main Application Setup ---
if __name__ == "__main__":
    logger.info("Starting Telegram Bot App...")
    setup_database()

    # Optional: Setup persistence for user_data (to remember selections across restarts)
    # You might need to install `python-telegram-bot[persistence]`
    # persistence = DictPersistence() # Simple in-memory dict persistence
    # Consider PicklePersistence for saving to file:
    # from telegram.ext import PicklePersistence
    # persistence = PicklePersistence(filepath='bot_persistence.pickle')

    # Create the Application and pass it your bot's token.
    application = Application.builder().token(TELEGRAM_TOKEN).build() # Add .persistence(persistence) if using it

    # Add command handlers
    application.add_handler(CommandHandler("start", start))

    # Add callback query handler
    application.add_handler(CallbackQueryHandler(handle_callback_query))

    # Add the repeating job to the queue
    job_queue = application.job_queue
    job_queue.run_repeating(send_updates_job, interval=UPDATE_INTERVAL_SECONDS, first=10, name="send_price_updates") # Start after 10 sec

    logger.info("Bot polling started...")
    # Run the bot until the user presses Ctrl-C
    application.run_polling(allowed_updates=Update.ALL_TYPES)

    logger.info("Bot stopped.")

2025-04-03 12:39:52,017 - INFO - Starting Telegram Bot App...
2025-04-03 12:39:52,017 - INFO - User database setup complete.
C:\Users\aliho\AppData\Local\Temp\ipykernel_17620\1596703924.py:413: PTBUserWarning: No `JobQueue` set up. To use `JobQueue`, you must install PTB via `pip install "python-telegram-bot[job-queue]"`.
  job_queue = application.job_queue


AttributeError: 'NoneType' object has no attribute 'run_repeating'